In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM


In [47]:

# Load the data
df = pd.read_csv('output.csv', header=None)

# Ignore the even lines
df = df[df.index % 2 != 0]

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df.values)

# Split the data into training and test sets
train_size = int(len(scaled_data) * 0.8)
train, test = scaled_data[0:train_size,:], scaled_data[train_size:len(scaled_data),:]


In [48]:

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        dataX.append(a)
        dataY.append(dataset[i + look_back, :])
    return np.array(dataX), np.array(dataY)


In [49]:

# Reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)


In [50]:

# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))




In [55]:
# Create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, 6)))
model.add(Dense(6))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)


Epoch 1/100
38/38 - 4s - loss: 0.2971 - 4s/epoch - 99ms/step
Epoch 2/100
38/38 - 0s - loss: 0.2505 - 149ms/epoch - 4ms/step
Epoch 3/100
38/38 - 0s - loss: 0.2100 - 142ms/epoch - 4ms/step
Epoch 4/100
38/38 - 0s - loss: 0.1735 - 144ms/epoch - 4ms/step
Epoch 5/100
38/38 - 0s - loss: 0.1419 - 141ms/epoch - 4ms/step
Epoch 6/100
38/38 - 0s - loss: 0.1171 - 138ms/epoch - 4ms/step
Epoch 7/100
38/38 - 0s - loss: 0.0984 - 135ms/epoch - 4ms/step
Epoch 8/100
38/38 - 0s - loss: 0.0857 - 136ms/epoch - 4ms/step
Epoch 9/100
38/38 - 0s - loss: 0.0776 - 136ms/epoch - 4ms/step
Epoch 10/100
38/38 - 0s - loss: 0.0733 - 136ms/epoch - 4ms/step
Epoch 11/100
38/38 - 0s - loss: 0.0699 - 137ms/epoch - 4ms/step
Epoch 12/100
38/38 - 0s - loss: 0.0682 - 136ms/epoch - 4ms/step
Epoch 13/100
38/38 - 0s - loss: 0.0672 - 136ms/epoch - 4ms/step
Epoch 14/100
38/38 - 0s - loss: 0.0668 - 137ms/epoch - 4ms/step
Epoch 15/100
38/38 - 0s - loss: 0.0663 - 138ms/epoch - 4ms/step
Epoch 16/100
38/38 - 0s - loss: 0.0661 - 153ms/epoc

In [56]:

next_numbers = model.predict(np.array([scaled_data[-1]]).reshape(1, 1, 6))
next_numbers = scaler.inverse_transform(next_numbers)



1/1 [==============================] - 1s 869ms/step


In [57]:
# Round the numbers and convert them to integers
next_numbers = [int(round(num)) for num in next_numbers[0]]

print(f'The predicted next set of numbers is {next_numbers}')


The predicted next set of numbers is [9, 16, 22, 24, 31, 35]
